<a href="https://colab.research.google.com/github/GruffGemini/ComputationalLinguistics/blob/main/%D0%94%D0%97_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

In [1]:
!pip install tokenizers 
!pip install ipython-autotime
%load_ext autotime

     |████████████████████████████████| 6.6 MB 8.7 MB/s 
time: 10.6 ms (started: 2022-04-28 12:09:44 +00:00)


In [2]:
from pprint import pprint

from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase
from tokenizers.trainers import WordPieceTrainer
from tokenizers import decoders

from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from tqdm.auto import tqdm

import numpy as np
import tensorflow as tf

time: 3.09 s (started: 2022-04-28 12:09:50 +00:00)


In [3]:
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/de-en/opus.de-en-train.de
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/de-en/opus.de-en-train.en
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/de-en/opus.de-en-test.de
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/de-en/opus.de-en-test.en

--2022-04-28 12:09:55--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/de-en/opus.de-en-train.de
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75998572 (72M)
Saving to: ‘opus.de-en-train.de’

opus.de-en-train.de 100%[===================>]  72.48M  19.7MB/s    in 4.6s    

2022-04-28 12:10:01 (15.7 MB/s) - ‘opus.de-en-train.de’ saved [75998572/75998572]

--2022-04-28 12:10:01--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/de-en/opus.de-en-train.en
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70247384 (67M)
Saving to: ‘opus.de-en-train.en’

opus.de-en-train.en 100%[===================>]  66.99M  19.5MB/s    in 4.4s    

2022-04-28 12:10:06 (15.2 MB/s) - ‘opus.d

In [4]:
de_sents = open('opus.de-en-train.de').read().lower().splitlines()
en_sents = open('opus.de-en-train.en').read().lower().splitlines()

time: 3.03 s (started: 2022-04-28 12:10:14 +00:00)


In [ ]:
def train_tokenizer(train_set_name: str):
  tokenizer = Tokenizer(WordPiece(), )
  tokenizer.normalizer = normalizers.Sequence([Lowercase()])
  tokenizer.pre_tokenizer = Whitespace()
  trainer = WordPieceTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]"])
  tokenizer.train(files=[train_set_name], trainer=trainer)
  return tokenizer

time: 5.04 ms (started: 2022-04-28 07:22:56 +00:00)


In [8]:
"""
tokenizer_de = train_tokenizer('opus.de-en-train.de')
tokenizer_en = train_tokenizer('opus.de-en-train.en')
tokenizer_de.save('tokenizer_de')
tokenizer_en.save('tokenizer_en')
"""
tokenizer_en = Tokenizer.from_file("drive/MyDrive/CompLing 9/tokenizer_en")
tokenizer_de = Tokenizer.from_file("drive/MyDrive/CompLing 9/tokenizer_de")

time: 816 ms (started: 2022-04-28 12:11:47 +00:00)


In [ ]:
def encode(text, tokenizer):
    return [tokenizer.token_to_id('[CLS]')] + tokenizer.encode(text).ids + [tokenizer.token_to_id('[SEP]')]

time: 1.24 ms (started: 2022-04-28 07:23:02 +00:00)


In [ ]:
X_en = [encode(t, tokenizer_en) for t in en_sents]
X_de = [encode(t, tokenizer_de) for t in de_sents]

time: 1min 1s (started: 2022-04-28 07:23:05 +00:00)


In [ ]:
max_len_de = int(np.mean([len(x) for x in X_de])) + 1
max_len_en = int(np.mean([len(x) for x in X_en])) + 1
PAD_IDX = tokenizer_en.token_to_id('[PAD]')

time: 348 ms (started: 2022-04-28 07:24:13 +00:00)


In [ ]:
X_de = tf.keras.preprocessing.sequence.pad_sequences(X_de, maxlen=max_len_de, padding='post', value=tokenizer_de.token_to_id('[PAD]'))

X_en_out = tf.keras.preprocessing.sequence.pad_sequences([x[1:] for x in X_en], maxlen=max_len_en-1, padding='post', value=tokenizer_de.token_to_id('[PAD]'))

X_en_dec = tf.keras.preprocessing.sequence.pad_sequences([x[:-1] for x in X_en], maxlen=max_len_en-1, padding='post', value=tokenizer_en.token_to_id('[PAD]'))

time: 16.5 s (started: 2022-04-28 07:24:29 +00:00)


In [ ]:
X_de_train, X_de_valid, X_en_dec_train, X_en_dec_valid, X_en_out_train, X_en_out_valid = train_test_split(X_de, X_en_dec, X_en_out, test_size=0.05)

time: 383 ms (started: 2022-04-28 07:24:48 +00:00)


In [ ]:
def scaled_dot_product_attention(query, key, value, mask):
    """Calculate the attention weights. """
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    # scale matmul_qk
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    # add the mask to zero out padding tokens
    if mask is not None:
        logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k)
    attention_weights = tf.nn.softmax(logits, axis=-1)

    output = tf.matmul(attention_weights, value)

    return output

time: 7.43 ms (started: 2022-04-28 07:24:52 +00:00)


In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # linear layers
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # split heads
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # scaled dot-product attention
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # concatenation of heads
        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # final linear layer
        outputs = self.dense(concat_attention)

        return outputs

time: 79 ms (started: 2022-04-28 07:24:54 +00:00)


In [ ]:
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, PAD_IDX), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]

time: 2.66 ms (started: 2022-04-28 07:24:57 +00:00)


In [ ]:
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

time: 4.19 ms (started: 2022-04-28 07:25:00 +00:00)


In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):

    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):

        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

time: 20.3 ms (started: 2022-04-28 07:25:02 +00:00)


In [ ]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

time: 12.2 ms (started: 2022-04-28 07:25:04 +00:00)


In [ ]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

time: 10.3 ms (started: 2022-04-28 07:25:07 +00:00)


In [ ]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
    attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

    attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

time: 22.7 ms (started: 2022-04-28 07:25:10 +00:00)


In [ ]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

time: 11.3 ms (started: 2022-04-28 07:25:13 +00:00)


In [ ]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                max_len,
                name="transformer"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

    enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)
    # mask the future tokens for decoder inputs at the 1st attention block
    look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
    # mask the encoder outputs for the 2nd attention block
    dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

    enc_outputs = encoder(
      vocab_size=vocab_size[0],
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      max_len=max_len[0],
    )(inputs=[inputs, enc_padding_mask])

    dec_outputs = decoder(
      vocab_size=vocab_size[1],
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      max_len=max_len[1],
    )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    outputs = tf.keras.layers.Dense(units=vocab_size[1], name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

time: 21.1 ms (started: 2022-04-28 07:25:16 +00:00)


In [ ]:
L  = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none',)

def loss_function(y_true, y_pred):
    loss = L(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, PAD_IDX), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

time: 3.36 ms (started: 2022-04-28 07:25:19 +00:00)


In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

time: 8.62 ms (started: 2022-04-28 07:25:21 +00:00)


In [ ]:
tf.keras.backend.clear_session()

NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8
UNITS = 512
DROPOUT = 0.1


mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = transformer(
        vocab_size=(tokenizer_de.get_vocab_size(),tokenizer_en.get_vocab_size()),
        num_layers=NUM_LAYERS,
        units=UNITS,
        d_model=D_MODEL,
        num_heads=NUM_HEADS,
        dropout=DROPOUT,
        max_len=[max_len_de, max_len_en])

    optimizer = tf.keras.optimizers.Adam(
        0.001, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    def accuracy(y_true, y_pred):
        return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


    model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
    checkpoint = tf.keras.callbacks.ModelCheckpoint('drive/MyDrive/CompLing 9/model_deen',
                                                monitor='val_loss',
                                                verbose=1,
                                            save_weights_only=True,
                                            save_best_only=True,
                                            mode='min',
                                            save_freq='epoch')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
time: 7.2 s (started: 2022-04-28 07:25:28 +00:00)


In [ ]:
'''model.fit((X_de_train, X_en_dec_train), X_en_out_train, 
             validation_data=((X_de_valid, X_en_dec_valid), X_en_out_valid),
             batch_size=200,
             epochs=10,
             callbacks=[checkpoint]
             )'''

Epoch 1/10
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
4750/4750 [==============================] - ETA: 0s - loss: 2.5217 - accuracy: 0.2974
Epoch 1: val_loss improved from inf to 1.99523, saving model t

In [ ]:
model.load_weights('drive/MyDrive/CompLing 9/model_deen')
#loss, acc = model.evaluate((X_de_valid, X_en_dec_valid), X_en_out_valid)

time: 8.56 s (started: 2022-04-28 07:25:40 +00:00)


In [ ]:
def translate(text):
    input_ids = encode(text.lower(), tokenizer_de)
    input_ids = tf.keras.preprocessing.sequence.pad_sequences([input_ids], maxlen=max_len_de, padding='post')
    output_ids = [tokenizer_en.token_to_id('[CLS]') ] 
    pred = model((input_ids, tf.cast([output_ids], tf.int32)), training=False) 

    while pred.numpy().argmax(2)[0][-1] not in [tokenizer_en.token_to_id('[SEP]')]:
      if len(output_ids) > max_len_en or not pred.numpy().argmax(2)[0][-1]:
        break
      output_ids.append(pred.numpy().argmax(2)[0][-1])
      pred = model((input_ids, tf.cast([output_ids], tf.int32)), training=False)

    return tokenizer_en.decode(output_ids[1:])

time: 9.53 ms (started: 2022-04-28 07:25:51 +00:00)


In [ ]:
def batch_translate(batch):
    input_ids = [encode(text.lower(), tokenizer_de) for text in batch]
    input_ids = tf.keras.preprocessing.sequence.pad_sequences(input_ids, maxlen=max_len_de, padding='post', truncating='post')
    output_ids = [[tokenizer_en.token_to_id('[CLS]')] for _ in range(len(input_ids))] 
    batch_pred = model((input_ids, tf.cast(output_ids, tf.int32)), training=False)

    while True:
      if all(
          pred.numpy().argmax(1)[-1] in [tokenizer_en.token_to_id('[SEP]')] or 
          len(output_id) >= max_len_en or 
          not pred.numpy().argmax(1)[-1]
          
          for output_id, pred in zip(output_ids, batch_pred)
        ):
        break
      for i in range(len(output_ids)):
        output_ids[i].append(batch_pred[i].numpy().argmax(1)[-1])
      batch_pred = model((input_ids, tf.cast(output_ids, tf.int32)), training=False) 

    return [tokenizer_en.decode(output_id[1:]) for output_id in output_ids]

time: 12.1 ms (started: 2022-04-28 07:25:56 +00:00)


In [ ]:
print(translate("du hast mich gefragt und ich hab' nichts gesagt"))
print(translate("Neunundneunzig Luftballons Auf ihrem Weg zum Horizont"))
print(translate("All diese Momente werden verloren sein in der Zeit, so wie Tränen im Regen"))

you asked me and i didn ' t say
nine and new air trainees on your way to the horizon
all these moments are lost in time , like tears in the rain
time: 6.6 s (started: 2022-04-28 07:25:58 +00:00)


In [ ]:
pprint(batch_translate(["du hast mich gefragt und ich hab' nichts gesagt",
                        "Neunundneunzig Luftballons Auf ihrem Weg zum Horizont",
                        "All diese Momente werden verloren sein in der Zeit, so wie Tränen im Regen"]))

["you asked me and i didn ' t say",
 'nine and new air trainees on your way to the horizon',
 'all these moments are lost in time , like tears in the rain']
time: 1.92 s (started: 2022-04-28 07:26:07 +00:00)


In [ ]:
import nltk
import pickle

BATCH_SIZE = 500
SKIP = 1200

translations = []
with open(f'drive/MyDrive/CompLing 9/translations_{SKIP}.pickle', 'rb') as f:
  translations = pickle.load(f)

for index, batch in enumerate(tqdm([de_sents[i:i + BATCH_SIZE] for i in range(0, len(de_sents), BATCH_SIZE)])):
  if index <= SKIP:
    continue
  try:
    translations.extend(batch_translate(batch))
  except Exception as e:
    print(e)
    print(batch)
  if not index % 100:
    filename = f'drive/MyDrive/CompLing 9/translations_{index}.pickle'
    with open(filename, 'wb') as f:
      pickle.dump(translations, f)
      print(f'translations saved as {filename}')

  0%|          | 0/2000 [00:00<?, ?it/s]

translations saved as drive/MyDrive/CompLing 9/translations_1300.pickle
translations saved as drive/MyDrive/CompLing 9/translations_1400.pickle
translations saved as drive/MyDrive/CompLing 9/translations_1500.pickle
translations saved as drive/MyDrive/CompLing 9/translations_1600.pickle
translations saved as drive/MyDrive/CompLing 9/translations_1700.pickle
translations saved as drive/MyDrive/CompLing 9/translations_1800.pickle
translations saved as drive/MyDrive/CompLing 9/translations_1900.pickle
time: 3h 41min 41s (started: 2022-04-28 07:29:30 +00:00)


In [ ]:
with open('drive/MyDrive/CompLing 9/translations', 'wb') as f:
  pickle.dump(translations, f)

time: 697 ms (started: 2022-04-28 11:12:05 +00:00)


In [5]:
import pickle
with open('drive/MyDrive/CompLing 9/translations', 'rb') as f:
  translations = pickle.load(f)

time: 935 ms (started: 2022-04-28 12:10:24 +00:00)


In [6]:
print(len(de_sents))
print(len(translations))
print(de_sents[4243])
print(translations[4243])

1000000
1000000
ich bitte dich, du kannst doch nicht ernsthaft in ihn verliebt sein, oder?
i beg you , can ' t you seriously fall in love with him , right ?
time: 131 ms (started: 2022-04-28 12:10:27 +00:00)


In [12]:
import nltk

bleus = []

for i, t in enumerate(translations):
  reference = tokenizer_en.encode(t).tokens
  hypothesis = tokenizer_en.encode(en_sents[i]).tokens

bleus.append(nltk.translate.bleu_score.sentence_bleu([reference], hypothesis,  ))

time: 44.2 s (started: 2022-04-28 12:15:14 +00:00)


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [13]:
(sum(bleus)/len(bleus))*100

20.28736642487601

time: 3.24 ms (started: 2022-04-28 12:16:12 +00:00)


# Задание 2

Прием backtranslation применяется в том случае, если имеется недостаточно параллельных текстов (то есть обучающая выборка, где каждому тексту языка-источника сопоставляется текст целевого языка, слишком мала). Backtranslation позволяет искусственно увеличить объем обучающей выборки путем генерации дополнительных пар текстов. 

Техника применима в том случае, если для целевого языка имеется достаточно большое число текстов (без соответствия на языке-источнике). В этом случае строится вспомогательная модель машинного перевода, обучаемая на имеющейся обучающей выборке, но в обратную сторону. То есть, модель обучается переводить тексты с целевого языка на язык-источник. Тогда, взяв сколь угодно много текстов на целевом языке, можно сгенерировать такое же большое число текстов на языке-источнике, после чего использовать полученные пары при обучении основной модели.

Если рассматривать пример из семинара, прием backtranslation был бы применим в том гипотетическом случае, если бы имелось много текстов на русском языке, но мало на английском. Тогда на имеющихся данных можно было бы обучить простой трансформер, который бы переводил предложения с русского на английский. Затем следовало бы из какого-либо другого источника получить множество текстов на русском языке и прогнать на них вспомогательный трансформер, получив столько же искусственных текстов на английском языке. Таким образом, было бы получено расширение обучающей выборки, и уже на этой расширенной выборке можно было бы обучать основной трансформер, осуществляющий перевод с английского на русский.